# Deep Learning Package - Keras(tensorflow)

---

### 케라스(Keras)

<img src="./data/keras_logo.png" width="600"/>

[Keras getting started](https://keras.io/getting_started/)

[User Guide](https://keras.io/guides/)

[API Reference](https://keras.io/api/)

[Examples](https://keras.io/examples/)


# 1 Keras 사용 예제

In [ ]:
#!pip install tensorflow

In [ ]:
#!pip install keras

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.rc('figure', figsize=(10, 6))

from matplotlib import rcParams
rcParams['font.family'] = 'New Gulim'
rcParams['font.size'] = 10
rcParams['axes.unicode_minus'] = False

### 1.1 보스턴 집값 예측 - Regression

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

import tensorflow as tf

In [ ]:
# seed 값 설정
np.random.seed(0)
tf.random.set_seed(3)

#### 데이터 로드

In [ ]:
df = pd.read_csv('./data/housing.csv', delim_whitespace=True, header=None)
dataset = df.values
X = dataset[:,0:13]
y = dataset[:,13]

#### 데이터 분할

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

#### 모델 생성 및 설정

In [ ]:
model_b = Sequential()
model_b.add(Dense(30, input_dim=13, activation='relu'))
model_b.add(Dense(6, activation='relu'))
model_b.add(Dense(1))

#### 모델 계층 확인

In [ ]:
model_b.summary()

#### 모델 컴파일

In [ ]:
model_b.compile(loss='mean_squared_error', optimizer='adam')

#### 모델 학습 실행

In [ ]:
%%time
model_b.fit(X_train, y_train, epochs=200, batch_size=10)

#### 결과 예측

In [ ]:
pred = model_b.predict(X_test).flatten()
pred

#### 결과 평가 - RMSE

In [ ]:
from sklearn import metrics

# RMSE(Root Mean Squared Error)
metrics.mean_squared_error(y_test, pred)**0.5

#### 최종 모델 저장

In [ ]:
model_b.save('./model/model_boston.h5')

#### 모델 불러오기

In [ ]:
from keras.models import load_model

model_boston_load = load_model('./model/model_boston.h5')

model_boston_load.summary()

### 1.2 와인 종류 예측 - Classification

[Data - UCI repository](https://archive.ics.uci.edu/ml/datasets/wine)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint,EarlyStopping

import tensorflow as tf

In [ ]:
# seed 값 설정
np.random.seed(3)
tf.random.set_seed(3)

#### 데이터 로드 - 20%

In [ ]:
df_pre = pd.read_csv('./data/wine.csv', header=None)
df = df_pre.sample(frac=0.2) # 20%

dataset = df.values
X = dataset[:,0:12]
y = dataset[:,12]

#### 데이터 분할

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

#### 모델 생성 및 설정

In [ ]:
model_w = Sequential()
model_w.add(Dense(30,  input_dim=12, activation='relu'))
model_w.add(Dense(12, activation='relu'))
model_w.add(Dense(8, activation='relu'))
model_w.add(Dense(1, activation='sigmoid'))

#### 모델 계층 확인

In [ ]:
model_w.summary()

#### 모델 컴파일

In [ ]:
model_w.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#### 모델 업데이트 및 저장 설정

In [ ]:
modelpath='./model/model_wine_{epoch:02d}-{val_loss:.4f}.hdf5'
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)

#### 학습 자동 중단 설정

In [ ]:
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=100)

#### 모델 학습 실행 및 저장

In [ ]:
%%time
history = model_w.fit(X_train, y_train, validation_split=0.2, epochs=3500, batch_size=500, verbose=0,
                      callbacks=[early_stopping_callback,checkpointer])

#### 학습 진행 과정
- history
 - loss: 훈련 손실값
 - accuracy: 훈련 정확도
 - val_loss: 검증 손실값
 - val_accuracy: 검증 정확도

In [ ]:
fig, loss_ax = plt.subplots()
acc_ax = loss_ax.twinx()

loss_ax.plot(history.history['loss'], 'y', label='train loss')
loss_ax.plot(history.history['val_loss'], 'r', label='validation loss')
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
loss_ax.legend(bbox_to_anchor=(1.25, 0.15))

acc_ax.plot(history.history['accuracy'], 'b', label='train accuracy')
acc_ax.plot(history.history['val_accuracy'], 'g', label='validation accuracy')
acc_ax.set_ylabel('accuracy')
acc_ax.legend(bbox_to_anchor=(1.05, 1))

plt.title('학습 진행에 따른 학습 데이터와 검증 데이터의 정확도')
plt.show()

#### 결과 예측

In [ ]:
# 예측 확률
pred_prob = model_w.predict(X_test)
pred_prob

In [ ]:
# 결과 예측
pred = np.where(pred_prob > 0.5, 1, 0).flatten()
pred

#### 결과 평가

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, pred))

#### 최종 모델 저장

In [ ]:
model_w.save('./model/model_wine.h5')

#### 모델 불러오기

In [ ]:
from keras.models import load_model

model_wine_load = load_model('./model/model_wine.h5')

model_wine_load.summary()

---

In [ ]:
# End of file